In [1]:
import csv

with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

In [2]:
import pandas as pd
df = pd.read_csv("checkins.csv")
df.head(3)

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22


In [4]:
df.shape[0]

396634

In [3]:
df_reduced = df.iloc[:10**5, :]

In [4]:
from sklearn.cluster import MeanShift
import numpy as np
from collections import Counter

In [5]:
clustering = MeanShift(bandwidth=0.1).fit(df_reduced.loc[:, ['latitude', 'longitude']].values)

In [6]:
clusters_to_left = [k  for k, v in Counter(clustering.labels_).items() if (v > 15)]

In [8]:
centers_to_left = []
for center_lat, center_long in clustering.cluster_centers_:
    if clustering.predict([[center_lat, center_long]])[0] in clusters_to_left:
            centers_to_left.append([center_lat, center_long])

In [9]:
office_locations = np.array([[33.751277, -118.188740], #(Los Angeles)
                             [25.867736, -80.324116],# (Miami)
                             [51.503016, -0.075479],# (London)
                             [52.378894, 4.885084], #(Amsterdam)
                             [39.366487, 117.036146],# (Beijing)
                             [-33.868457, 151.205134]])#(Sydney)

In [27]:
best_points=np.ndarray(shape=(0,2))
dist=[]
for point in centers_to_left:
    best_dist=1000000
    best_point = None
    
    for office in office_locations:
        d = np.sqrt((point[0]-office[0])**2+(point[1]-office[1])**2)
        if d<best_dist:
            best_dist=d
            best_point = point
    dist.append(best_dist)
    best_points = np.vstack([best_points, best_point])

dist = np.array(dist)
idxs = np.argsort(dist)[:20]

In [30]:
points=np.array(centers_to_left)
for point,dis in zip(best_points[idxs], dist[idxs]):
    print("dist=%f; %f,%f" % (dis, point[0], point[1]))

dist=0.007835; -33.860630,151.204776
dist=0.009353; 52.372964,4.892317
dist=0.022674; 25.845672,-80.318891
dist=0.050058; 51.502991,-0.125537
dist=0.070848; 33.809878,-118.148924
dist=0.134109; 25.785812,-80.217938
dist=0.167406; 25.705350,-80.283429
dist=0.188876; 26.010098,-80.199991
dist=0.195779; 33.888325,-118.048928
dist=0.211811; 33.872986,-118.362091
dist=0.222233; 33.972575,-118.168371
dist=0.271301; 26.138844,-80.334347
dist=0.294979; 33.983936,-118.007405
dist=0.302270; 26.120863,-80.158907
dist=0.304731; 33.817306,-117.891249
dist=0.314884; 34.060398,-118.248709
dist=0.338810; 33.674303,-117.858789
dist=0.340846; 26.200585,-80.250716
dist=0.378688; 34.035487,-118.438998
dist=0.386706; 34.131460,-118.118012


In [35]:
with open("ans.txt", "w") as f:
    f.write("-33.860630 151.204776")